## Data Wrangling: League of Legends Ranked Games

##### Import Module / Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

##### Loading CSV files into dataframes

In [162]:
champs = 'champs.csv'
champs_df = pd.read_csv(champs)

matches = 'matches.csv'
matches_df = pd.read_csv(matches)

participants = 'participants.csv'
participants_df = pd.read_csv(participants)

stats1 = 'stats1.csv'
stats1_df = pd.read_csv(stats1)

stats2 = 'stats2.csv'
stats2_df = pd.read_csv(stats2, low_memory=False)

bans = 'teambans.csv'
bans_df = pd.read_csv(bans)

teamstats = 'teamstats.csv'
teamstats_df = pd.read_csv(teamstats)

### Prepare Matches Dataframe

In [194]:
matches_df.head(2)

,id,gameid,platformid,queueid,seasonid,duration,creation,version
0,10,3187427022,EUW1,420,8,1909,1495068946860,7.10.187.9675
1,11,3187425281,EUW1,420,8,1693,1495066760778,7.10.187.9675


In [195]:
matches_df.duration = matches_df.duration * .01  # Convert duration to minutes

In [196]:
conditions = [ # Create list of current values
        matches_df.platformid == 'EUN1',
        matches_df.platformid == 'EUW1',
        matches_df.platformid == 'NA1',
        matches_df.platformid == 'TR1']
choices = ['Europe North', 'Europe West', 'North America', 'Turkey'] # Create list of new values

matches = matches_df[['platformid', 'seasonid', 'duration']].copy()

In [197]:
matches.platformid = np.select(conditions, choices) # Map conditions with choices
matches = matches.rename(index = str, columns = {"platformid": "Region","duration":"Duration","seasonid":'Season'}) #Rename columns

In [198]:
matches.head(2)

,Region,Season,Duration
0,Europe West,8,19.09
1,Europe West,8,16.93


### Preparing Participants Dataframe

In [199]:
participants_df.head(2) # Explore dataframe

,id,matchid,player,championid,ss1,ss2,role,position
0,9,10,1,19,4,11,NONE,JUNGLE
1,10,10,2,267,3,4,DUO_SUPPORT,BOT


In [200]:
participants_merged = pd.merge(participants_df, # Merge champions dataframe with participants dataframe
                               champs_df,how='inner', left_on='championid'
                               , right_on='id').sort_values(by =['matchid'])

participants_columns = participants_merged[['matchid', # Select columns
                                            'player',
                                            'position',
                                            'name']] 

participants = participants_columns.rename(index = str, #Rename columns
                                           columns = {"matchid": "Match ID",
                                                      "player":"Player",
                                                      "position":'Position',
                                                      "name": 'Champion'}) 

participants = participants.reset_index() # Reset index

participants = participants[['Match ID', 'Player', 'Position', 'Champion']] # Reselect columns


In [201]:
participants.head(2) # Confirm preparation

,Match ID,Player,Position,Champion
0,10,1,JUNGLE,Warwick
1,10,3,BOT,Draven


### Preparing Team Bans

In [191]:
bans_df.head(2)

,matchid,teamid,championid,banturn
0,10,100,11,1
1,10,100,117,3


In [184]:
bans_merged = pd.merge(bans_df, # Merge champions dataframe with bans dataframe
                               champs_df,how='inner', left_on='championid'
                               , right_on='id').sort_values(by =['matchid'])
bans_columns = bans_merged[['matchid', # Select columns
                                            'banturn',
                                            'name']] 
bans = bans_columns.rename(index = str, #Rename columns
                                           columns = {"matchid": "Match ID",
                                                      "banturn":"Ban Turn",
                                                      "name": 'Champion'})

In [185]:
bans = bans.reset_index() # Reset index
bans = bans[['Match ID', 'Ban Turn', 'Champion']] # Reselect columns after resetting index

In [186]:
bans.head(2)

,Match ID,Ban Turn,Champion
0,10,1,Master Yi
1,10,6,Fizz


### Preparing Individual Stats

In [205]:
stats1_df.head(2) # Explore stats1 dataframe

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,3748,2003,3111,3053,1419,1042,3340,6,...,69,42,27,610,13,0,0,10,0,0
1,10,0,2301,3111,3190,3107,0,0,3364,0,...,1,1,0,211,14,1,0,17,3,0


In [152]:
stats2_df.head(2) # Explore stats2 dataframe

,id,win,item1,item2,item3,item4,item5,item6,trinket,kills,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,1028382,0,1056,3001,1052,3020,1058,1026,3340,7,...,0,0,0,50,12,0,0,6,0,0
1,1028383,0,1041,2003,0,0,0,0,3340,0,...,11,11,0,114,3,0,0,0,0,0


In [164]:
stats = stats1_df.append([stats2_df]) # Union stats dataframes
stats.shape # Confirm union

(1834517, 56)

In [173]:
stats = stats[['id', 'win', 'kills', # Select Columns
               'neutralminionskilled','ownjunglekills','enemyjunglekills',
               'totcctimedealt', 'champlvl', 'pinksbought','wardsbought','wardsplaced','wardskilled','firstblood']]

In [209]:
stats.head(10)

,id,win,kills,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
0,9,0,6,69,42,27,610,13,0,0,10,0,0
1,10,0,0,1,1,0,211,14,1,0,17,3,0
2,11,0,7,3,1,2,182,14,1,0,13,5,0
3,12,0,5,6,6,0,106,15,0,0,3,0,0
4,13,0,2,4,3,1,159,15,0,0,10,0,0
5,14,1,3,111,81,30,808,16,0,0,8,2,0
6,15,1,4,6,0,6,179,16,1,0,7,0,0
7,16,1,13,6,3,3,48,17,0,0,8,0,0
8,17,1,15,7,2,5,440,16,1,0,6,1,1
9,18,1,4,2,1,1,260,16,5,0,25,7,0


### Preparing Team Stats

In [207]:
teamstats_df.head(2)

,matchid,teamid,firstblood,firsttower,firstinhib,firstbaron,firstdragon,firstharry,towerkills,inhibkills,baronkills,dragonkills,harrykills
0,10,100,0,1,0,0,0,0,5,0,0,0,0
1,10,200,1,0,1,1,1,1,10,3,1,3,1
